In [36]:
from abr.Abr import ABR
from arbre234.Arbre234 import Arbre234
from arbreB.ArbreB import ArbreB
from arbreDST.ArbreDST import ArbreDST
from arbreSplay.ArbreSplay import ArbreSplay
from arbreAVL.ArbreAVL import ArbreAVL
from fileBinomiale.FileBinomiale import FileBinomiale
from rtrie.RTrie import RTrie
from trieBinaire.TrieBinaire import TrieBinaire
from trieHybride.TrieHybride import TrieHybride
from tasMin.TasMin import TasMin


# Table des matières <a class="anchor" id="menu"></a>
* [Tas Min](#tasmin)
* [File Binomiale](#filebino)
* [Arbre Binaire de Recherche (ABR)](#abr)
* [Arbre 2-3-4](#arbre234)
* [Arbre-B](#arbreb)
* [Arbre AVL](#avl)
* [Arbre Auto-adaptatif (Splay-tree)](#splaytree)
* [Arbre Digital (DST)](#dst)
* [Arbre Lexicographique](#lexico)
* [R-Trie](#rtrie)
* [Trie Hybride](#triehybr)

___
## Tas Min <a class="anchor" id="tasmin"></a>                                                                                                                                  
#### [Retour](#menu) 



In [1]:
from graphviz import Digraph

class TasMin:
    def __init__(self):
        self.tas = [] 

    # Ajout
    
    def ajout_liste(self, cles: list) -> 'TasMin':
        '''Tas * list -> Tas'''
        '''Ajoute une liste de clés dans le tas et retourne un nouveau tas'''
        nouveauTas = self
        for cle in cles:
            nouveauTas = nouveauTas.ajout(cle)
        return nouveauTas
            
    def ajout(self, cle : int) -> 'TasMin':
        '''Tas * int -> Tas'''
        '''Ajoute une clé dans le tas et retourne un nouveau tas'''
        nouveauTas = TasMin()
        nouveauTas.tas = self.tas.copy()
        nouveauTas.tas = nouveauTas._ajout(cle)
        return nouveauTas
    
    def _ajout(self, cle : int) -> list:
        '''Tas * int -> list'''
        '''Ajoute une clé dans le tas et retourne le tas modifié'''
        self.tas.append(cle)  # add the new key
        i = len(self.tas) - 1
        # moves it up (swap with parent) until it satisfies the tas property
        while (i != 0) and self.tas[i] < self.tas[self._parent(i)]:
            self.tas[i], self.tas[self._parent(i)] = (
                self.tas[self._parent(i)], self.tas[i])
            i = self._parent(i)
        return self.tas

    # SupprMin
    def extractMin(self) -> tuple[int, 'TasMin']:
        '''Tas -> (int * Tas)'''
        '''Retourne la clé minimum et le tas sans le minimum '''
        if self.tas[0] is None:
            return None
        nouveauTas = TasMin()
        nouveauTas.tas = self.tas.copy()
        nouveauTas.tas[0], nouveauTas.tas[-1] = nouveauTas.tas[-1], nouveauTas.tas[0]
        racine = nouveauTas.tas.pop()
        nouveauTas._heapify(0)
        return racine, nouveauTas

    # Construction
    def construction(self, cles: list) -> 'TasMin':
        '''Tas * list -> Tas'''
        '''Construit un tas à partir d'une liste de clés et retourne un nouveau tas'''
        self.tas = cles
        n = len(self.tas)
        # opti = loop half : nodes, not leaves. second half contains leaves(=node without children)
        for i in range(n//2, -1, -1):
            self._heapify(i)
        return self

    # Union
    def union(tas1: 'TasMin', tas2: 'TasMin') -> 'TasMin':
        '''Tas * Tas -> Tas'''
        '''Retourne un nouveau tas qui est l'union de deux tas'''
        nouveauTas = TasMin()
        valeurs = tas1.tas + tas2.tas
        nouveauTas.construction(valeurs)
        return nouveauTas
    
    
    
    # Helper functions
    def _est_tas(self) -> bool: 
        '''Tas -> bool'''
        '''Retourne True si le tas est un tas, False sinon'''
        n = len(self.tas)
        for i in range(0, n//2):
            gauche = 2*i + 1
            droite = 2*i + 2
            if gauche < n and self.tas[gauche] < self.tas[i]:
                return False
            if droite < n and self.tas[droite] < self.tas[i]:
                return False
        return True
    
    def _parent(self, i: int) -> int:  
        '''Tas * int -> int'''
        '''Retourne l'index du parent de l'élément à l'index i'''
        # the floor division // rounds the result down to the nearest whole number
        return (i-1)//2

    # moves down the key at index i until it satisfies the tas property
    def _heapify(self, i: int) -> None:  
        '''Tas * int -> None'''
        '''Modifie le tas pour satisfaire la propriété de tas'''
        min = i
        gauche = 2 * i + 1
        droite = 2 * i + 2
        if gauche < len(self.tas) and self.tas[gauche] < self.tas[min]:
            min = gauche
        if droite < len(self.tas) and self.tas[droite] < self.tas[min]:
            min = droite
        if min != i:
            self.tas[i], self.tas[min] = (self.tas[min], self.tas[i])
            self._heapify(min)
            
            
    def visualiser_arbre(self):
        '''Tas -> dot'''
        '''Affiche le tas'''
        dot = Digraph()
    
        def traverse(index):
            if index < len(self.tas):
                dot.node(str(self.tas[index]), str(self.tas[index]))
                
                gauche = 2 * index + 1
                droite = 2 * index + 2
                
                if gauche < len(self.tas):
                    dot.edge(str(self.tas[index]), str(self.tas[gauche]), label="L", style="dashed")
                    traverse(gauche)
                
                if droite < len(self.tas):
                    dot.edge(str(self.tas[index]), str(self.tas[droite]), label="R", style="solid")
                    traverse(droite)
        
        traverse(0)
        return dot


## File Binomiale <a class="anchor" id="filebino"></a>

In [ ]:
from collections import deque
import copy

In [2]:
from graphviz import Digraph
from collections import deque
import copy

class ArbreBinomial:
    def __init__(self, cle = None):
        self.cle = cle
        self.parent = None
        self.enfants = deque()
        self.degree = 0
    
    def estVide(self) -> bool:
        '''ArbreBinomial -> bool'''
        '''Verifie si l'arbre est vide ou non'''
        return self.cle is None
    
    def union_arbre(self, arbre: 'ArbreBinomial') -> 'ArbreBinomial':
        '''ArbreBinomial * ArbreBinomial -> ArbreBinomial'''
        '''Union de deux arbres binomiaux de même degré'''
        if (self.degree != arbre.degree):
            raise ValueError("Les arbres n'ont pas le même degré")
        if self.cle < arbre.cle:
            nouveauArbre = ArbreBinomial(self.cle)
            nouveauArbre.enfants = copy.deepcopy(self.enfants)
            nouveauArbre.enfants.appendleft(arbre)
            nouveauArbre.parent = self
            nouveauArbre.degree = self.degree + 1
            return nouveauArbre
        else:
            nouveauArbre = ArbreBinomial(arbre.cle)
            nouveauArbre.enfants = copy.deepcopy(arbre.enfants)
            nouveauArbre.enfants.appendleft(self)
            nouveauArbre.parent = arbre
            nouveauArbre.degree = arbre.degree + 1
            return nouveauArbre

    def decapitate(self) -> 'FileBinomiale':
        '''ArbreBinomial -> FileBinomiale'''
        '''Retire la racine de l'arbre et renvoie un file contenant les sous-arbres'''
        file = FileBinomiale()
        for enfant in self.enfants:
            enfant.parent = None
            file.arbres.append(enfant)
        return file

    def transformation_file(self) -> 'FileBinomiale':
        '''ArbreBinomial -> FileBinomiale'''
        '''Transforme un arbre en file'''
        file = FileBinomiale()
        file.arbres = [self]
        return file

class FileBinomiale:
    def __init__(self):
        self.arbres = deque()  # liste d'arbres binomiaux

    #AJOUT
    def ajout_liste(self, liste: list) -> 'FileBinomiale':
        '''FileBinomiale * list -> FileBinomiale'''
        '''Ajoute une liste de clés à la file'''
        file = self
        for cle in liste:
            file = file.ajout(cle)
        return file
    
    def ajout(self, cle) -> 'FileBinomiale':
        '''FileBinomiale * int -> FileBinomiale'''
        '''Ajoute une clé à la file'''
        nouvelleFile = FileBinomiale()
        nouvelleFile.arbres = copy.deepcopy(self.arbres)
        resultat = nouvelleFile.unionFile(ArbreBinomial(cle).transformation_file())
        nouvelleFile.arbres = resultat.arbres
        return resultat
    
    #SUPPRESSION
    def extractMin(self) -> tuple[int, 'ArbreSplay']:
        '''FileBinomiale -> int * FileBinomiale'''
        '''Supprime et renvoie la clé minimale de la file'''
        nouvelleFile = FileBinomiale()
        nouvelleFile.arbres = copy.deepcopy(self.arbres)
        if self.estVide():
            return None
        arbreMin = nouvelleFile.arbres[0]
        for arbre in nouvelleFile.arbres:
            if arbre.cle < arbreMin.cle:
                arbreMin = arbre
        nouvelleFile.arbres.remove(arbreMin)
        if (len(nouvelleFile.arbres) == 0):
            nouvelleFile.arbres = arbreMin.decapitate().arbres
        else:
            nouvelleFile.arbres = FileBinomiale.unionFile(nouvelleFile, arbreMin.decapitate()).arbres
        return arbreMin.cle, nouvelleFile
    
    #UNION
    def unionFile(self, file: 'FileBinomiale') -> 'FileBinomiale':
        '''FileBinomiale * FileBinomiale -> FileBinomiale'''
        '''Union de deux files binomiales'''
        nouvelleFile = FileBinomiale()
        nouvelleFile.arbres = copy.deepcopy(self.arbres)
        resultat = FileBinomiale._union(nouvelleFile, file, ArbreBinomial())
        nouvelleFile.arbres = copy.deepcopy(resultat.arbres)
        return nouvelleFile

    def _union(file1: 'FileBinomiale', file2: 'FileBinomiale', arbre: 'ArbreBinomial') -> 'FileBinomiale':
        '''FileBinomiale * FileBinomiale * ArbreBinomial -> FileBinomiale'''
        '''Fonction auxiliaire pour l'union de deux files binomiales'''
        if arbre.estVide():
            if file1.estVide():
                return file2
            if file2.estVide():
                return file1

            arbre1 = file1.minDeg()
            arbre2 = file2.minDeg()

            if (arbre1.degree < arbre2.degree):
                return FileBinomiale._ajoutMin(FileBinomiale.unionFile(file1._reste(), file2), arbre1)
            if (arbre2.degree < arbre1.degree):
                return FileBinomiale._ajoutMin(FileBinomiale.unionFile(file1, file2._reste()), arbre2)
            if (arbre1.degree == arbre2.degree):
                return FileBinomiale._union(file1._reste(), file2._reste(), ArbreBinomial.union_arbre(arbre1, arbre2))
        else:
            if (file1.estVide()):
                return FileBinomiale.unionFile(file2, arbre.transformation_file())
            if (file2.estVide()):
                return FileBinomiale.unionFile(file1, arbre.transformation_file())

            arbre1 = file1.minDeg()
            arbre2 = file2.minDeg()

            if (arbre.degree < arbre1.degree and arbre.degree < arbre2.degree):
                return FileBinomiale._ajoutMin(FileBinomiale.unionFile(file1, file2), arbre)
            if (arbre.degree == arbre1.degree and arbre.degree == arbre2.degree):
                return FileBinomiale._ajoutMin(FileBinomiale._union(file1._reste(), file2._reste(), ArbreBinomial.union_arbre(arbre1, arbre2)), arbre)
            if (arbre.degree == arbre1.degree and arbre.degree < arbre2.degree):
                return FileBinomiale._union(file1._reste(), file2, ArbreBinomial.union_arbre(arbre1, arbre))
            if (arbre.degree == arbre2.degree and arbre.degree < arbre1.degree):
                return FileBinomiale._union(file1, file2._reste(), ArbreBinomial.union_arbre(arbre2, arbre))
            
    
    #FONCTIONS UTILES / PRIMITIVES
    def estVide(self) -> bool:
        '''FileBinomiale -> bool'''
        '''Verifie si la file est vide ou non'''
        return self is None or len(self.arbres) == 0

    def minDeg(self) -> 'ArbreBinomial':
        '''FileBinomiale -> ArbreBinomial'''
        '''Renvoie l'arbre de degré minimum de la file'''
        minDeg = self.arbres[0]
        for arbre in self.arbres:
            if arbre.degree < minDeg.degree:
                minDeg = arbre
        return minDeg
    
    def _reste(self) -> 'FileBinomiale':
        '''FileBinomiale -> FileBinomiale'''
        '''Retourne la file sans l'arbre de degré minimum'''
        minDeg = self.minDeg()
        self.arbres.remove(minDeg)
        return self
        
    def _ajoutMin(self, arbre: 'ArbreBinomial') -> 'FileBinomiale':
        '''FileBinomiale * ArbreBinomial -> FileBinomiale'''
        '''Ajoute un arbre à la file'''
        nouvelleFile = FileBinomiale()
        nouvelleFile.arbres = copy.deepcopy(self.arbres)
        nouvelleFile.arbres.append(arbre)
        return nouvelleFile
    
    def visualiser_arbre(self):
        dot = Digraph()

        def traverse(arbre):
            if arbre is None:
                return

            dot.node(str(arbre.cle), label=str(arbre.cle))

            for enfant in arbre.enfants:
                dot.edge(str(arbre.cle), str(enfant.cle))
                traverse(enfant)

        for arbre in self.arbres:
            traverse(arbre)

        return dot

## Arbre Binaire de Recherche (ABR) <a class="anchor" id="abr"></a>

In [3]:
from graphviz import Digraph

class ArbreBinaire:
    def __init__(self, cle: int, gauche = None, droite = None) -> None:
        self.cle = cle
        self.gauche = gauche
        self.droite = droite


class ABR:
    def __init__(self) -> None:
        self.racine = None

    def est_vide(self) -> bool:
        return self.racine is None

    #RECHERCHE
    def recherche(self, cle : int) -> bool:
        '''ABR * int -> bool'''
        '''Recherche une clé dans l'arbre binaire'''
        return self._recherche(self.racine, cle)
    
    def _recherche(self, noeud : 'ArbreBinaire', cle : int) -> bool:
        '''ABR * ArbreBinaire * int -> bool'''
        '''Recherche une clé dans l'arbre binaire'''
        if noeud is None:
            return False
        elif cle < noeud.cle:
            return self._recherche(noeud.gauche, cle)
        elif cle > noeud.cle:
            return self._recherche(noeud.droite, cle)
        else:
            return True

    ## AJOUT
    
    def ajout_liste(self, cles : list) -> 'ABR':
        '''ABR * list -> ArbreBinaire'''
        '''Ajoute une liste de clés à l'arbre binaire'''
        abr = ABR()
        for cle in cles:
            abr = abr.ajout(cle)
        return abr
    
    def ajout(self, cle : int) -> 'ABR' :
        '''ABR * int -> ArbreBinaire'''
        '''Ajoute une clé à l'arbre binaire'''
        abr = ABR()
        abr.racine = self._ajout(self.racine, cle)
        return abr

    def _ajout(self, noeud : ArbreBinaire, cle : int) -> ArbreBinaire :
        '''ABR * ArbreBinaire * int -> ArbreBinaire'''
        '''Ajoute une clé à l'arbre binaire'''
        if noeud is None: # si l'arbre est vide, on crée un nouveau noeud 
            return ArbreBinaire(cle)
        elif cle == noeud.cle: # si la clé est déjà dans l'arbre, on ne fait rien 
            return ArbreBinaire(noeud.cle, noeud.gauche, noeud.droite, noeud.hauteur)
        elif cle < noeud.cle: # si la clé est inférieure à la clé du noeud, on ajoute la clé dans le sous-arbre gauche
            nouveauGauche = self._ajout(noeud.gauche, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, nouveauGauche, noeud.droite)
        else: # si la clé est supérieure à la clé du noeud, on ajoute la clé dans le sous-arbre droit
            nouveauDroite = self._ajout(noeud.droite, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, noeud.gauche, nouveauDroite)

        return nouveauNoeud
    
    #SUPPRESSION
    def supprime(self, cle : int) -> 'ABR':
        '''ABR * int -> ArbreBinaire'''
        '''Supprime une clé de l'arbre binaire'''
        abr = ABR()
        abr.racine = self._supprime(self.racine, cle)
        return abr
    
    def _supprime(self, noeud : 'ArbreBinaire', cle : int) -> 'ArbreBinaire':
        '''ABR * ArbreBinaire * int -> ArbreBinaire'''
        '''Supprime une clé de l'arbre binaire'''
        if noeud is None: # si l'arbre est vide, on ne fait rien
            return noeud
        elif cle < noeud.cle: # si la clé est inférieure à la clé du noeud, on supprime la clé dans le sous-arbre gauche
            nouveauNoeud = ArbreBinaire(noeud.cle, self._supprime(noeud.gauche, cle), noeud.droite)
        elif cle > noeud.cle: # si la clé est supérieure à la clé du noeud, on supprime la clé dans le sous-arbre droit
            nouveauNoeud = ArbreBinaire(noeud.cle, noeud.gauche, self._supprime(noeud.droite, cle))
        else: # sinon la clé est égale à la clé du noeud, on supprime le noeud et on le remplace par le sous-arbre gauche ou droit
            # si un des sous-arbres est vide, on remplace le noeud par l'autre sous-arbre
            if noeud.gauche is None: 
                return noeud.droite
            elif noeud.droite is None: 
                return noeud.gauche

            # sinon, on remplace le noeud par le noeud minimal du sous-arbre droit
            min_noeud = self._min_noeud(noeud.droite)
            nouveauNoeud = ArbreBinaire(min_noeud.cle, noeud.gauche, self._supprime(noeud.droite, min_noeud.cle))

        return nouveauNoeud
    
    def _min_noeud(self, noeud: ArbreBinaire) -> ArbreBinaire:
        '''ABR * ArbreBinaire -> ArbreBinaire'''
        '''Retourne le noeud minimal = le plus à gauche de l'arbre binaire'''
        if noeud is None or noeud.gauche is None:
            return noeud
        return self._min_noeud(noeud.gauche)
    
    def visualiser_arbre(self) -> None:
        '''ABR -> None'''
        '''Affiche l'arbre binaire'''
        dot = Digraph()
        def traverse(noeud):
            if noeud is None:
                return

            dot.node(str(noeud.cle), str(noeud.cle))
            
            if noeud.gauche is not None:
                dot.edge(str(noeud.cle), str(noeud.gauche.cle), label="L", style="dashed")
                traverse(noeud.gauche)
            
            if noeud.droite is not None:
                dot.edge(str(noeud.cle), str(noeud.droite.cle), label="R", style="solid")
                traverse(noeud.droite)
        traverse(self.racine)
        return dot

## Arbre 2-3-4 <a class="anchor" id="arbre234"></a>

In [4]:
from graphviz import Digraph
from bisect import bisect_left

class Noeud234:
    def __init__(self, cles, arbres=None, parent=None):
        self.cles = cles      # Liste des clés
        self.arbres = arbres  # Liste des sous-arbres
        self.parent = parent
        
        
class Arbre234:
    def __init__(self, noeud=None):
        self.racine = noeud
        self.ecrGauche = None
        # sert à savoir si l'ajout continue dans le fils gauche ou dans le fils droit après éclatement
        # True pour le fils gauche sinon c'est le fils droit


    def ajout(self, x):
        '''Arbre234 * int -> Arbre234
        Renvoie l'arbre après l'ajout de x'''
        if self._est_vide():
            return Arbre234(Noeud234([x]))
        
        return self._ajout(x)
    

    def recherche(self, x):
        '''Arbre234 * int -> Arbre234
        Renvoie l'arbre contenant x sinon None'''
        if self._est_feuille() and not self._contient(x):
            return None
        
        if self._contient(x):
            return self
        
        if x < self._contenu()[0]:
            return self._sous_arbres()[0].Recherche(x)
        elif self._degre() == 2 or x < self._contenu()[1]:
            return self._sous_arbres()[1].Recherche(x)
        elif self._degre() == 3 or x < self._contenu()[2]:
            return self._sous_arbres()[2].Recherche(x)
        else:
            return self._sous_arbres()[3].Recherche(x)
        

    def supprime(self, x):
        '''Arbre234 * int -> Arbre234
        Renvoie l'arbre après la suppression de x'''
        if self._est_vide():
            return Arbre234()
            
        if self._est_feuille():
            if self._degre() == 2 and self._contient(x):
                return Arbre234()
        
        else: # Cas où la fusion est nécessaire entre la racine et ses fils
            G = self._sous_arbres()[0]
            D = self._sous_arbres()[1]
            if self._degre() == 2 and G._degre() == 2 and D._degre() == 2:
                self.racine = self._fusion(G, D)

        return self._supprime(x)
    

    def construction(self, liste):
        '''Arbre234 * list -> Arbre234
        Renvoie l'arbre après l'ajout des éléments de la liste'''
        for x in liste:
            self = self.ajout(x)
        return self
    

    def afficher_arbre(self, niveau=0):
        '''Arbre234 * int -> None
        Affiche l'arbre dans le terminal'''
        if self.racine is not None:
            print("  " * niveau, self._contenu())
            if not self._est_feuille():
                for sous_arbre in self._sous_arbres():
                    sous_arbre.afficher_arbre(niveau + 1)


    def visualiser_arbre(self):
        '''Arbre234 -> None
        Visualiser l'arbre avec Graphviz'''
        def creer_label_noeud(A):
            '''Arbre234 -> str'''
            noeud_label = ""
            sections = []

            # Commencer par un espace vide pour les sous-arbres
            sections.append(f' <f0> ')

            # Ajouter les clés et les espaces vides pour les sous-arbres entre chaque clé
            for i, cle in enumerate(A._contenu()):
                sections.append(f' <f{i*2+1}> {str(cle)} ')
                sections.append(f' <f{i*2+2}> ')

            noeud_label = '|'.join(sections)
            return noeud_label
        
        def ajouter_noeuds_et_arcs(A, graph, parent_id=None, parent_port=''):
            '''Arbre234 * Digraph * str * str -> None'''
            if A._est_vide():
                return
            
            noeud_label = creer_label_noeud(A)
            noeud_id = f'node{abs(hash(A)) % (10 ** 8)}'  # Génère un ID unique plus court
            graph.node(noeud_id, label=noeud_label)

            if parent_id is not None and parent_port != '':
                graph.edge(f'{parent_id}:{parent_port}', noeud_id)

            if A._est_feuille():
                return

            sous_arbres = A._sous_arbres()
            for i, arbre in enumerate(sous_arbres):
                port = f'f{i*2}'  # Connecter aux ports des sous-arbres
                ajouter_noeuds_et_arcs(arbre, graph, noeud_id, port)

        g = Digraph('G', node_attr={'shape': 'record', 'height': '.1'})
        ajouter_noeuds_et_arcs(self, g)
        return g
    
    def _est_vide(self):
        '''Arbre234 -> bool'''
        return self.racine is None


    def _est_feuille(self):
        '''Arbre234 -> bool'''
        return self._sous_arbres() is None


    def _contenu(self):
        '''Arbre234 -> list[int]
        Retourne la liste des clés de l'arbre'''
        return self.racine.cles


    def _sous_arbres(self):
        '''Arbre234 -> list[Arbre234]
        Retourne la liste des sous-arbres de l'arbre'''
        return self.racine.arbres
        

    def _pere(self):
        '''Arbre234 -> Arbre234'''
        return self.racine.parent


    def _degre(self):
        '''Arbre234 -> int'''
        return len(self._contenu()) + 1


    def _contient(self, x):
        '''Arbre234 * int -> bool'''
        return x in self._contenu()
        

    def _index(self):
        '''Arbre234 -> int
        Retourne la position de l'arbre dans son père'''
        if self._pere() is None:
            return -1
        
        return self._pere()._sous_arbres().index(self)


    def _duplique(self, parent=None):
        '''Arbre234 -> Arbre234
        Retourne une copie de l'arbre'''
        if self._est_feuille():
            return Arbre234(Noeud234(self._contenu().copy(), parent=parent))
        
        A = Arbre234()
        sous_arbres = []
        contenus = self._contenu().copy()
        for i in range(self._degre()):
            sous_arbres.append(self._sous_arbres()[i]._duplique(A))
        A.racine = Noeud234(contenus, sous_arbres, parent)
        return A


    def _insertion_triee(self, x):
        '''Arbre234 * int -> None
        Insère x dans la liste des clés de l'arbre'''
        indice = bisect_left(self._contenu(), x)
        self._contenu().insert(indice, x)
        return indice
    
    def _ajout(self, x, parent=None):
        '''Arbre234 * int * Arbre234-> Arbre234'''
        if self._contient(x):
            return self._duplique(parent)
        
        if self._degre() == 4:
            self.racine = self._eclatement_descente(x, parent)

        if self._est_feuille():
            A = self._duplique(parent)
            A._insertion_triee(x)
            return A
        
        A = Arbre234(Noeud234(self._contenu().copy(), [], parent))
        indice = bisect_left(A._contenu(), x)

        # On duplique les sous arbres sauf celui qui va être modifié
        for (i, a) in enumerate(self._sous_arbres()):
            if i != indice:
                A._sous_arbres().append(a._duplique(A))
            else:
                SousA = a._ajout(x, A)
                if A.ecrGauche:
                    A.ecrGauche = None # Insère x dans le fils gauche après éclatement
                    A._sous_arbres().insert(indice, SousA)
                else:
                    A._sous_arbres().append(SousA) # Insère x dans le fils droit après éclatement       
        return A

    def _eclatement_descente(self, x, parent=None):
        '''Arbre234 -> Noeud234'''
        G = Arbre234()
        D = Arbre234()

        if self._est_feuille():
            G.racine = Noeud234([self._contenu()[0]], parent=parent)
            D.racine = Noeud234([self._contenu()[2]], parent=parent)

        else:
            G.racine = Noeud234([self._contenu()[0]], [self._sous_arbres()[0], self._sous_arbres()[1]], parent=parent)
            D.racine = Noeud234([self._contenu()[2]], [self._sous_arbres()[2], self._sous_arbres()[3]], parent=parent)
            self._sous_arbres()[0].racine.parent = G
            self._sous_arbres()[1].racine.parent = G
            self._sous_arbres()[2].racine.parent = D
            self._sous_arbres()[3].racine.parent = D

        if parent is None: # Eclatement à la racine
            return Noeud234([self._contenu()[1]], [G, D])
        
        # Eclatement d'un noeud autre que la racine
        indice = parent._insertion_triee(self._contenu()[1])
        if x < self._contenu()[1]:
            D.racine.parent = parent
            parent._sous_arbres().insert(indice, D)
            parent.ecrGauche = True
            return G.racine
        
        G.racine.parent = parent
        parent._sous_arbres().insert(indice, G)
        return D.racine
    
    def _supprime(self, x, parent=None):
        '''Arbre234 * int -> Arbre234'''
        if self._est_feuille():
            if self._contient(x):
                if self._degre() > 2:
                    A = self._duplique(parent)
                    A._contenu().remove(x)
                    return A
                
                else:
                    VG = self._frere_gauche()
                    VD = self._frere_droit()
                    if VG is None:
                        if VD is None: # Cas où l'arbre est la racine avec qu'un élément, déjà traité dans supprime
                            pass
                        
                        else:
                            if VD._degre() > 2: # Cas où self est le plus à gauche et qu'il peut emprunter à son voisin droit
                                monte = VD._cle_minimale()
                                self._emprunt(VD, 0, monte)

                            else:  # Cas où self est le plus à gauche et qu'il ne peut pas emprunter
                                self.racine = parent._fusion(self, VD, 0)

                    else:
                        if VG._degre() > 2: # Cas où self peut emprunter à son voisin gauche
                            monte = VG._cle_maximale()
                            self._emprunt(VG, VG._index(), monte)

                        else:
                            if VD is None: # Cas où self est le plus à droite et qu'il ne peut pas emprunter
                                self.racine = parent._fusion(VG, self, VG._index())

                            else:
                                if VD._degre() > 2: # Cas où self peut emprunter à son voisin droit et qu'il ne peut pas emprunter à son voisin gauche
                                    monte = VD._cle_minimale()
                                    self._emprunt(VD, self._index(), monte)

                                else: # Cas où self ne peut pas emprunter à ses voisins
                                    self.racine = parent._fusion(VG, self, VG._index())
                                    
                    return self._supprime(x, parent)
                            
            else: # x n'a pas été trouvé
                return self._duplique(parent)
        
        else:
            A = Arbre234(Noeud234(self._contenu().copy(), [], parent))
            if self._contient(x): # x se trouve dans un noeud interne
                indice = A._contenu().index(x)
                for (_, a) in enumerate(self._sous_arbres()):
                    A._sous_arbres().append(a._duplique(A))

                G = A._sous_arbres()[indice]
                D = A._sous_arbres()[indice + 1]
                if not G._est_arbre_minimal():
                    # la suppression est possible dans le sous_arbre gauche
                    # et on supprime la clé maximale, qui se trouve dans les feuilles
                    max = G._cle_maximale()
                    A._contenu()[indice] = max
                    A._sous_arbres()[indice] = G._supprime(max, A)

                elif not D._est_arbre_minimal():
                    # idem pour le sous_arbre droit
                    min = D._cle_minimale()
                    A._contenu()[indice] = min
                    A._sous_arbres()[indice + 1] = D._supprime(min, A)

                else: # Impossible d'emprunter il faut fusionner
                    D.racine = A._fusion(G, D, indice)
                    A._sous_arbres()[indice] = D._supprime(x, A)

            else:
                indice = bisect_left(A._contenu(), x)
                indiceNonMinimal = None # récupérer l'indice d'un sous-arbre non minimal
                for (i, a) in enumerate(self._sous_arbres()):
                    if i != indice:
                        A._sous_arbres().append(a._duplique(A))
                        if not a._est_arbre_minimal():
                            indiceNonMinimal = i
                    else:
                        a.racine.parent = A
                        A._sous_arbres().append(a)
                
                sousA = A._sous_arbres()[indice]
                if sousA._est_arbre_minimal() and not sousA._est_feuille():
                    if indiceNonMinimal is None:
                        # Tous minimal, on fusionne donc les 2 premiers sous-arbres
                        A._sous_arbres()[0].racine = A._fusion(A._sous_arbres()[0], A._sous_arbres()[1], 0)
                        return A._supprime(x, parent)

                    if indiceNonMinimal > indice:
                        # self se trouve à gauche de l'arbre non minimal, on transfère une clé de droite à gauche
                        for i in range(indiceNonMinimal, indice, -1):
                            VD = A._sous_arbres()[i]
                            min = VD._cle_minimale()
                            cleATransferer = A._contenu()[i-1]
                            A._sous_arbres()[i] = VD._supprime(min, A)
                            A._contenu()[i-1] = min
                            A._sous_arbres()[i-1] = A._sous_arbres()[i-1]._ajout(cleATransferer, A)._supprime(x, A)
                    else:
                        # self se trouve à droite de l'arbre non minimal, on transfère une clé de gauche à droite
                        for i in range(indiceNonMinimal, indice):
                            VG = A._sous_arbres()[i]
                            max = VG._cle_maximale()
                            cleATransferer = A._contenu()[i]
                            A._sous_arbres()[i] = VG._supprime(max, A)
                            A._contenu()[i] = max
                            A._sous_arbres()[i+1] = A._sous_arbres()[i+1]._ajout(cleATransferer, A)._supprime(x, A)
                    return A      

                # Cas récursif
                sousA = A._sous_arbres()[indice]._supprime(x, A)
                indice = bisect_left(A._contenu(), sousA._cle_minimale()) # On a retiré un arbre lors de l'éclatement
                A._sous_arbres()[indice] = sousA

            return A


    def _emprunt(self, V, i, monte):
        '''Arbre234 * Arbre234 * int * int -> None
        Transfère une clé de V à self'''
        descend = self._pere()._contenu()[i]
        V._contenu().remove(monte)
        self._pere()._contenu()[i] = monte
        self._contenu().append(descend)


    def _est_arbre_minimal(self):
        '''Arbre234 -> bool
        Retourne True si l'arbre est minimal, aucun n'emprunt n'est possible dans self, False sinon'''
        if self._est_feuille():
            return self._degre() == 2
        else:
            if self._degre() == 2:
                return self._sous_arbres()[0]._est_arbre_minimal() and self._sous_arbres()[1]._est_arbre_minimal()
            else:
                return False
            

    def _fusion(self, G, D, i=None):
        '''Arbre234 * Arbre234 * Arbre234 * int -> Noeud234
        Fusionne G et D en un seul arbre + la clé qui les sépare'''
        g = G._contenu()[0]
        d = D._contenu()[0]
        sous_arbres = None

        if not G._est_feuille():
            sous_arbres = [G._sous_arbres()[0], G._sous_arbres()[1], D._sous_arbres()[0], D._sous_arbres()[1]]

        if self._pere() is None and self._degre() == 2:
            return Noeud234([g, self._contenu()[0], d], sous_arbres)
        
        p = self._contenu()[i]
        self._sous_arbres().remove(G)
        self._contenu().remove(p)
        return Noeud234([g, p, d], sous_arbres, self)


    def _cle_minimale(self):
        '''Arbre234 -> int
        Retourne la plus petite clé de l'arbre'''
        if self._est_feuille():
            return self._contenu()[0]
        
        return self._sous_arbres()[0]._cle_minimale()


    def _cle_maximale(self):
        '''Arbre234 -> int
        Retourne la plus grande clé de l'arbre'''
        if self._est_feuille():
            return self._contenu()[self._degre() - 2]
        
        return self._sous_arbres()[self._degre() - 1]._cle_maximale()


    def _frere_gauche(self):
        '''Arbre234 -> Arbre234'''
        position = self._index()

        if position == 0 or position == -1:
            return None
        
        return self._pere()._sous_arbres()[position - 1]


    def _frere_droit(self):
        '''Arbre234 -> Arbre234'''
        position = self._index()
        lePlusADroite = self._pere()._degre() - 1

        if position == lePlusADroite or position == -1:
            return None
        
        return self._pere()._sous_arbres()[position + 1]

## Arbre-B <a class="anchor" id="arbreb"></a>

In [ ]:
from graphviz import Digraph, nohtml
from bisect import insort_left, bisect_left
from math import floor

class PageB:
    def __init__(self, cles:'list'=[]):
        self.cles = cles # nbCles noeud : entre m et 2m, nbCles racine : entre 1 et 2m
        self.enfants = []
        self.parent = None

    def _est_page_vide(self):
        """ Retourne true si la page self est vide (sans cle).
        """
        return self.cles == []

    def _est_page_externe(self):
        """ Retourne true si self est une page externe (feuille).
        """
        return len(self.enfants) == 0

    def _est_debordement_page(self, nbMaxCles):
        """ Retourne true si la page self depasse sa limite de cles (2*ordre).    
        """
        return len(self.cles) > nbMaxCles
    
    def _est_suppression_sans_deficit(self, nbMinCles):
        """ Retourne true si la suppression d'une cle de la page self n'entraine pas
            de deficit en cle ie. si len(self.cles) > nbMinCles (racine : minimum 1 cle
            et noeud : minimum m cles, avec m = ordre de l'arbreB).
        """
        if self.parent is None: # racine : nbCles min = 1
            return len(self.cles) > 1
        else: # noeud : nbCles min = m
            return len(self.cles) > nbMinCles
    
    def _est_page_en_deficit(self, nbMinCles):
        """ Retourne true si la page self est en deficit de cle ie. si len(self.cles) < nbMinCles.
        """
        if self.parent is None: # racine : nbCles min = 1
            return len(self.cles) < 1
        else: # noeud : nbCles min = m
            return len(self.cles) < nbMinCles

    def _recherche_page(self, cle):
        """ Retourne true si cle est dans la page self.
        """
        return cle in self.cles
    
    def _duplique_page(self, parent:'PageB'=None):
        """ Retourne le duplique de la page self.
        """
        duplique = PageB(self.cles.copy())
        duplique.parent = parent
        for enfant in self.enfants:
            duplique.enfants.append(enfant._duplique_page(parent=duplique))
        return duplique


    def _indice_page_suivante(self, cle):
        """ Retourne l'indice de la page suivante (enfant de la page self) qui pourrait
            contenir la cle cle.
            Hypothese : self n'est pas une page externe (sinon pas de page suivante)
        """
        assert not self._est_page_externe(), "indicePageSuivante : self est une page externe, pas de page suivante!"
        indice = bisect_left(self.cles, cle)
        return indice
    
    def _frere_gauche(self, indicePage) -> 'PageB':
        """ indicePage : indice de la page self parmi les enfants de son parent.\n
            Retourne le frere gauche de la page self s'il existe,
            None si self ne possede pas frere gauche (self est
            la racine ou l'enfant le plus a gauche de son parent).
        """
        if self.parent is None or indicePage == 0: # self est la racine ou n'a pas de frere gauche
            return None
        else:
            return self.parent.enfants[indicePage - 1]
            
    def _frere_droit(self, indicePage) -> 'PageB':
        """ indicePage : indice de la page self parmi les enfants de son parent.\n
            Retourne le frere droit de la page self s'il exitse,
            None si self ne possede pas de frere droit (self est
            la racine ou l'enfant le plus a droite de son parent).
        """
        if self.parent is None or indicePage == len(self.parent.enfants) - 1: # self est la racine ou n'a pas de frere droit
            return None
        else:
            return self.parent.enfants[indicePage + 1]

    def _emprunter_cle_frere(self, frere:'PageB', indiceCle, estGauche:bool):
        """ Emprunte une cle de son frere gauche ou droit. Echange la
            cle empruntee avec une cle du parent pour respecter l'ordre des cles.
            La cle du parent a echanger est celle qui est a position indiceCle parmi
            les cles du parent.
            Si self n'est pas une feuille, elle recupere un enfant du frere (sinon
            deficit enfants).
            Hypothese : l'emprunt d'une cle de son frere n'entraine pas
            de deficit.
        """
        cleParent = self.parent.cles.pop(indiceCle) # cle du parent a echanger pour respecter l'ordre
        if estGauche: # emprunt au frere gauche
            cleEmpruntee = frere.cles.pop(len(frere.cles)-1) # emprunter la derniere cle du frere gauche
            self.cles.insert(0, cleParent) # la cle du parent devient la plus petite cle de self
            if not self._est_page_externe(): # si self n'est pas une feuille, recuperer un enfant du frere
                enfantFrere = frere.enfants.pop(len(frere.enfants)-1) # frere gauche cede son dernier enfant a self
                enfantFrere.parent = self
                self.enfants.insert(0, enfantFrere) # l'enfant du frere gauche devient le 1er enfant de self
        
        else: # emprunt au frere droit
            cleEmpruntee = frere.cles.pop(0) # emprunter la 1ere cle du frere droit
            self.cles.append(cleParent) # la cle du parent devient la plus grande cle de self
            if not self._est_page_externe():
                enfantFrere = frere.enfants.pop(0) # frere droit cede son 1er enfant a self
                enfantFrere.parent = self
                self.enfants.append(enfantFrere) # l'enfant du frere droit devient le dernier enfant de self
        self.parent.cles.insert(indiceCle, cleEmpruntee) # ajouter au parent la cle empruntee


    def _cle_predecesseur_dans_enfant(self):
        """ Fonction auxiliaire récursive de _suppression, retourne la cle predecesseur
            de cle dans l'enfant de page.
        """
        if self._est_page_externe(): # feuille : recuperer la cle max
            return self.cles[len(self.cles)-1]
        else: # noeud interne : continuer a chercher dans le sous-arbre droit
            return self.enfants[len(self.enfants)-1]._cle_predecesseur_dans_enfant()


    def _compensation_deficit(self, nbMinCles, indicePage):
        """ Compense le deficit de cles de la page self en empruntant une cle
            a son frere gauche ou droite si cette operation est possible, sinon
            fusionne la page self avec son frere gauche ou droite.
            Hypothese : self est en deficit de cles.
        """
        nouvelIndice = indicePage
        frereGauche = self._frere_gauche(nouvelIndice)
        frereDroit = self._frere_droit(nouvelIndice)
        if frereGauche is not None and frereGauche._est_suppression_sans_deficit(nbMinCles):
            self._emprunter_cle_frere(frereGauche, nouvelIndice-1, estGauche=True) # gauche -> indice cle enfant gauche
        elif frereDroit is not None and frereDroit._est_suppression_sans_deficit(nbMinCles):
            self._emprunter_cle_frere(frereDroit, nouvelIndice, estGauche=False) # droite -> indice cle enfant self
        else: # cas fusion
            if frereGauche is not None:
                self = self._fusion_page(frereGauche, nouvelIndice-1, estGauche=True)
                nouvelIndice = nouvelIndice - 1 # nouvelIndice = celui du frere gauche
            elif frereDroit is not None:
                self = self._fusion_page(frereDroit, nouvelIndice, estGauche=False)
        return self, nouvelIndice


    def _ajout_page_externe(self, cle):
        """ Ajoute la cle cle dans la page externe self.
        """
        assert self._est_page_externe(), "ajoutPageExterne : pas une page externe!"
        insort_left(self.cles, cle)

    def _eclatement(self, indicePage):
        """ indicePage : indice de la page self parmi les enfants de son parent, -1 si pas de parent.\n
            Eclate la page courante self en deux pages. La cle mediane est
            recuperee par le parent de self et les enfants de self sont
            partages entre les deux nouvelles pages.
            Si self n'a pas de parent (cas de la racine), une nouvelle
            racine est creee.
            Retourne soit le parent de self, soit la nouvelle racine.
            Hypothese : eclatement si len(self.cles) = 2*ordre+1 (debordement)
        """

        moitie = floor(len(self.cles)/2)
        cleMediane = self.cles[moitie]
        pageMoitieGauche = PageB(self.cles[:moitie]) # indice moitie pas inclus
        pageMoitieDroite = PageB(self.cles[moitie+1:]) # indice moitie+1 inclus

        if not self._est_page_externe(): # nb enfants = nb cles+1
            pageMoitieGauche.enfants = self.enfants[:moitie+1] # les nouvelles pages recuperent
            pageMoitieDroite.enfants = self.enfants[moitie+1:] # les enfants de self
            for page in self.enfants[:moitie+1]:
                page.parent = pageMoitieGauche
            for page in self.enfants[moitie+1:]:
                page.parent = pageMoitieDroite

        if self.parent is None: # self est racine -> creer une nouvelle racine
            nvRacine = PageB([cleMediane])
            nvRacine.enfants = [pageMoitieGauche, pageMoitieDroite]
            pageMoitieGauche.parent = nvRacine
            pageMoitieDroite.parent = nvRacine
            self.parent = nvRacine
            return nvRacine
        else: # un parent
            insort_left(self.parent.cles, cleMediane) # le parent recupere cle mediane
            self.parent.enfants.pop(indicePage)
            self.parent.enfants.insert(indicePage, pageMoitieGauche)
            self.parent.enfants.insert(indicePage+1, pageMoitieDroite)
            pageMoitieGauche.parent = self.parent
            pageMoitieDroite.parent = self.parent
            return self.parent


    def _fusion_page(self, frere:'PageB', indiceCle, estGauche:bool):
        """ Fusionne la page self avec une page frere gauche en ajoutant les cles de
            self dans celles de frere. Descend une cle du parent pour respecter
            l'ordre des cles dans l'ArbreB. Recupere les enfants de self et du frere.
            Retourne la page resultante.
        """
        fusion = PageB() # page contenant la fusion des cles de frere et self
        fusion.parent = self.parent
        if estGauche:
            cleParent = fusion.parent.cles.pop(indiceCle) # cle a descendre (et a supprimer du parent)
            fusion.cles = frere.cles + [cleParent] + self.cles
            fusion.enfants = frere.enfants + self.enfants # recuperer les enfants
        else: # frere droite
            cleParent = fusion.parent.cles.pop(indiceCle) # cle a descendre (et a supprimer du parent)
            fusion.cles = self.cles + [cleParent] + frere.cles
            fusion.enfants = self.enfants + frere.enfants # recuperer les enfants
        for enfant in fusion.enfants: # mettre a jour le nouveau parent des enfants
            enfant.parent = fusion
        fusion.parent.enfants.pop(indiceCle) # retirer self des enfants du parent
        fusion.parent.enfants.pop(indiceCle) # retirer frere des enfants du parent
        fusion.parent.enfants.insert(indiceCle, fusion) # ajouter la page resultante dans les enfants du parent
        return fusion
    

        

class ArbreB:
    def __init__(self, ordre):
        assert ordre > 0, "ArbreB : veuillez choisir un ordre strictement positif!"
        self.racine = None
        self.ordre = ordre # ordre = nombre de cles min dans un noeud non racine
        self.NBCLESMAX = 2*ordre
    
    def est_vide(self):
        return self.racine is None
    
    def _duplique(self):
        """ Retourne le duplique de l'ArbreB self.
        """
        if self.est_vide():
            return self
        duplique = ArbreB(self.ordre)
        duplique.racine = self.racine._duplique_page()
        return duplique
 
    def recherche(self, cle):
        """ Retourne true si cle est dans l'arbre-B self en
            cherchant dans les pagesB de l'arbre-B a l'aide
            de la fonction _recherche(pageB, cle).
        """
        if self.est_vide():
            return False
        return self._recherche(self.racine, cle)
    
    def _recherche(self, page:PageB, cle):
        """ Retourne true si la cle est dans l'arbre-B self en
            cherchant si la cle est dans une pageB de l'arbre.
        """
        if page._recherche_page(cle): # cle est dans la page courante
            return True
        else: # sinon continuer a chercher
            if page._est_page_externe(): # pas d'enfant : fin recherche
                return False
            else: # chercher dans la page suivante qui pourrait contenir la cle
                pageSuivante = page.enfants[page._indice_page_suivante(cle)]
                return self._recherche(pageSuivante, cle)


    def ajout(self, cle):
        if self.est_vide():
            self.racine = PageB([cle])
        else:
            self.racine, _ = self._ajout(self.racine, cle)


    def _ajout(self, page:PageB, cle, parent:PageB=None, indicePage=-1) -> tuple[PageB, bool]:
        """ indicePage : indice de la page parmi les enfants de son parent, -1 si pas de parent.\n
            Retourne la page resultante de l'ajout de la cle dans la page et un booleen indiquant
            si un eclatement a eu lieu (dans ce cas, la page retournee est le parent).
        """
        if page._recherche_page(cle): # cle deja dans une page de arbre-B
            return page, False
        
        nouvellePage = PageB(page.cles.copy()) # copier la page courante
        nouvellePage.parent = parent
        if page._est_page_externe(): # page externe (feuille) : ajout ok            
            nouvellePage._ajout_page_externe(cle)

        else: # ajout dans page suivante (enfant)
            indice = page._indice_page_suivante(cle) # indice de la page suivante (enfant) ou ajouter la cle
            for i in range(len(page.enfants)): # copier tous les enfants sauf celui ou on va ajouter la cle
                if i == indice: # ajouter temporairement l'ancien enfant (necessaire si eclatement)
                    nouvellePage.enfants.append(page.enfants[i])
                else:
                    nouvelEnfant = page.enfants[i]._duplique_page(parent=nouvellePage)
                    nouvellePage.enfants.append(nouvelEnfant)
            pageSuivante = nouvellePage.enfants[indice] # page enfant ou ajouter la cle
            pageSuivante, eclatement = self._ajout(pageSuivante, cle, nouvellePage, indice)
            if eclatement:
                nouvellePage = pageSuivante
            else:
                nouvellePage.enfants[indice] = pageSuivante # inserer l'enfant ou on a ajoute la cle

        # eclatement (retourne le parent de la page eclatee ou la nouvelle racine si on eclate la racine)
        if nouvellePage._est_debordement_page(self.NBCLESMAX):
            nouveauParent = nouvellePage._eclatement(indicePage)
            if nouveauParent.parent is None: # cas augmentation hauteur
                self.racine = nouveauParent # mise a jour de la racine
            return nouveauParent, True
        else:
            return nouvellePage, False

                    

    def suppression(self, cle):
        if self.est_vide():
            return
        else:
            self.racine, _ = self._suppression(self.racine, cle)


    def _suppression(self, page:PageB, cle, parent:PageB=None, indicePage=-1) -> tuple[PageB, int]:
        """ indicePage : indice de la page parmi les enfants de son parent, -1 si pas de parent.\n
            Retourne la page resultante de la suppression de la cle dans la page et l'indice
            de la page parmi les enfants de son parent.
            Si la cle a supprimer se trouve au niveau d'un noeud interne, elle est remplacee par sa
            cle predecesseur dans l'enfant du noeud interne et la fonction fait un appel recursif
            pour supprimer la cle predecesseur.
            Si suite a une suppression, la page est en deficit de cles, la fonction fait appel a
            _compensation_deficit pour emprunter une cle a un frere ou fusionner avec un frere.
        """
        if page._recherche_page(cle): # on a trouve la cle a supprimer
            if parent is None and len(page.cles) == 1 and page.enfants == []: # arbreB = 1 cle a la racine
                return None, -1
            
            nouvellePage = PageB(page.cles.copy()) # copier la page courante
            nouvellePage.parent = parent
            if page._est_page_externe(): # suppression au niveau d'une feuille : ok
                nouvellePage.cles.remove(cle)

            else: # sinon, suppression au niveau d'un noeud interne
                indice = nouvellePage.cles.index(cle)
                pagePrecedente = page.enfants[indice] # page contenant la cle predecesseur de cle
                for i in range(len(page.enfants)):
                    if i == indice:
                        nouvellePage.enfants.append(pagePrecedente)
                    else:
                        nouvelEnfant = page.enfants[i]._duplique_page(parent=nouvellePage)
                        nouvellePage.enfants.append(nouvelEnfant)

                cleRemplacement = pagePrecedente._cle_predecesseur_dans_enfant()
                nouvellePage.cles[indice] = cleRemplacement # echanger cle a supprimer par cle predecesseur
                # supprimer la cle predecesseur dans l'enfant pagePrecedente
                pagePrecedente, nouvelIndice = self._suppression(pagePrecedente, cleRemplacement, nouvellePage, indice)
                nouvellePage = pagePrecedente.parent # mise a jour de nouvellePage
                nouvellePage.enfants[nouvelIndice] = pagePrecedente # ok, on retourne None que pour le cas racine avec 1 cle
        
        else: # sinon, continuer a chercher la cle a supprimer
            if page._est_page_externe(): # cle a supprimer non trouvee dans l'arbre-B
                return page, indicePage # indicePage ne change pas
            else:
                nouvellePage = PageB(page.cles.copy()) # copier la page courante
                nouvellePage.parent = parent
                indice = page._indice_page_suivante(cle)
                pageSuivante = page.enfants[indice] # page suivante ou chercher la cle a supprimer
                for i in range(len(page.enfants)):
                    if i == indice:
                        nouvellePage.enfants.append(pageSuivante)
                    else:
                        nouvelEnfant = page.enfants[i]._duplique_page(parent=nouvellePage)
                        nouvellePage.enfants.append(nouvelEnfant)

                pageSuivante, nouvelIndice = self._suppression(pageSuivante, cle, nouvellePage, indice)
                nouvellePage = pageSuivante.parent # mise a jour de nouvellePage 
                nouvellePage.enfants[nouvelIndice] = pageSuivante

        # verifier si apres suppression, la page est en deficit de cles
        nouvelIndicePage = indicePage
        if nouvellePage.parent is None and nouvellePage._est_page_vide() and len(nouvellePage.enfants) == 1:
            # si nouvellePage = racine mais pas de cle apres avoir cede une cle a un de ses enfants
            # (apres une fusion), cas diminution hauteur -> nouvelle racine = enfant unique de nouvellePage
            self.racine = nouvellePage.enfants[0] # mise a jour de la racine
            return nouvellePage.enfants[0], -1
                  
        if nouvellePage._est_page_en_deficit(self.ordre): # deficit : emprunt ou fusion
            nouvellePage, nouvelIndicePage = nouvellePage._compensation_deficit(self.ordre, nouvelIndicePage)
        return nouvellePage, nouvelIndicePage
    
    def visualiser_arbre(arbreB):
        dot = Digraph(node_attr={'shape': 'record', 'height': '0.1', 'width': '0.02'})
        index = 0

        def bufferNoeud(cles):
            buffer = "<f0> |"
            for i in range(len(cles)):
                buffer += "<f" + str(2*i+1) + "> " + str(cles[i]) + "|" # case cle : indice impair
                if i != len(cles)-1:
                    buffer += "<f" + str(2*i+2) + "> " + "|" # case pointeur vers enfant : indice pair
                else:
                    buffer += "<f" + str(2*i+2) + "> "
            return buffer
        
        def traverse(pageB):
            nonlocal index

            if pageB is None:
                return

            buffer = bufferNoeud(pageB.cles)
            dot.node(str(index), nohtml(buffer))
            
            idParent = index
            for i in range(len(pageB.enfants)):
                parent = str(idParent) + ":f" + str(2*i)
                index += 1
                enfant = str(index)
                dot.edge(parent, enfant)
                traverse(pageB.enfants[i])
        
        traverse(arbreB.racine)
        return dot

## Arbre AVL <a class="anchor" id="avl"></a>

In [5]:
from graphviz import Digraph

class ArbreBinaire:
    def __init__(self, cle, gauche=None, droite=None, hauteur=1):
        self.cle = cle
        self.gauche = gauche
        self.droite = droite
        self.hauteur = hauteur

class ArbreAVL:
    def __init__(self):
        self.racine = None
    
    def _duplique(self, noeud) -> 'ArbreBinaire':
        if noeud is None:
            return None
        else:
            duplique = ArbreBinaire(noeud.cle, self._duplique(noeud.gauche), self._duplique(noeud.droite))
            duplique.hauteur = noeud.hauteur
            return duplique
    
    def ajout_liste(self, cles: list) -> 'ArbreAVL':
        avl = self
        for cle in cles:
            avl = avl.ajout(cle)
        return avl
    
    def ajout(self, cle: int) -> 'ArbreAVL':
        avl = ArbreAVL()
        avl.racine = self._duplique(self.racine)
        avl.racine = avl._ajout(avl.racine, cle)
        return avl
    
    def _ajout(self, noeud: ArbreBinaire, cle: int) -> ArbreBinaire:
        if not noeud:
            return ArbreBinaire(cle)
        elif cle < noeud.cle:
            noeud.gauche = self._ajout(noeud.gauche, cle)
        else:
            noeud.droite = self._ajout(noeud.droite, cle)
        
        noeud.hauteur = 1 + max(self._hauteur(noeud.gauche), self._hauteur(noeud.droite))
        return self._equilibrage(noeud, cle)
    
    def supprime(self, cle: int) -> 'ArbreAVL':
        avl = ArbreAVL()
        avl.racine = self._duplique(self.racine)
        avl.racine = avl._supprime(avl.racine, cle)
        return avl
    
    def _supprime(self, noeud: ArbreBinaire, cle: int) -> ArbreBinaire:
        if not noeud:
            return noeud
        elif cle < noeud.cle:
            noeud.gauche = self._supprime(noeud.gauche, cle)
        elif cle > noeud.cle:
            noeud.droite = self._supprime(noeud.droite, cle)
        else:
            if noeud.gauche is None:
                temp = noeud.droite
                noeud = None
                return temp
            elif noeud.droite is None:
                temp = noeud.gauche
                noeud = None
                return temp
            
            min_noeud = self._min_noeud(noeud.droite)
            noeud.cle = min_noeud.cle
            noeud.droite = self._supprime(noeud.droite, noeud.cle)
        
        if noeud is None:
            return noeud
        
        noeud.hauteur = 1 + max(self._hauteur(noeud.gauche), self._hauteur(noeud.droite))
        return self._equilibrage(noeud, cle)
    
    def recherche(self, cle: int) -> tuple[bool, 'ArbreBinaire']:
        return self._recherche(self.racine, cle)
    
    def _recherche(self, noeud: ArbreBinaire, cle: int) -> tuple[bool, 'ArbreBinaire']:
        if noeud is None:
            return False, None
        elif cle < noeud.cle:
            return self._recherche(noeud.gauche, cle)
        elif cle > noeud.cle:
            return self._recherche(noeud.droite, cle)
        else:
            return True, noeud
    
    def hauteur(self) -> int:
        return self._hauteur(self.racine)
    
    def _hauteur(self, noeud: ArbreBinaire) -> int:
        if not noeud:
            return 0
        
        return noeud.hauteur
    
    def _min_noeud(self, noeud: ArbreBinaire) -> ArbreBinaire:
        if noeud is None or noeud.gauche is None:
            return noeud
        return self._min_noeud(noeud.gauche)
    
    def _equilibre(self, noeud: ArbreBinaire) -> int:
        if not noeud:
            return 0
        return self._hauteur(noeud.gauche) - self._hauteur(noeud.droite)
    
    def _equilibrage(self, noeud: ArbreBinaire, cle: int) -> ArbreBinaire:
        equilibrage = self._equilibre(noeud)
        if equilibrage > 1:
            if self._equilibre(noeud.gauche)>=0:
                return self._rotation_droite(noeud)
            else:
                noeud.gauche = self._rotation_gauche(noeud.gauche)
                return self._rotation_droite(noeud)
        elif equilibrage < -1:
            if  self._equilibre(noeud.droite)<=0:
                return self._rotation_gauche(noeud)
            else:
                noeud.droite = self._rotation_droite(noeud.droite)
                return self._rotation_gauche(noeud)
        return noeud
    
    def _rotation_gauche(self, noeud: ArbreBinaire) -> ArbreBinaire:
        y = noeud.droite
        T2 = y.gauche
        y.gauche = noeud
        noeud.droite = T2
        noeud.hauteur = 1 + max(self._hauteur(noeud.gauche), self._hauteur(noeud.droite))
        y.hauteur = 1 + max(self._hauteur(y.gauche), self._hauteur(y.droite))
        return y
    
    def _rotation_droite(self, noeud: ArbreBinaire) -> ArbreBinaire:
        y = noeud.gauche
        T3 = y.droite
        y.droite = noeud
        noeud.gauche = T3
        noeud.hauteur = 1 + max(self._hauteur(noeud.gauche), self._hauteur(noeud.droite))
        y.hauteur = 1 + max(self._hauteur(y.gauche), self._hauteur(y.droite))
        return y
    
    def visualiser_arbre(self):
        dot = Digraph()
    
        def traverse(noeud):
            if noeud is None:
                return
            dot.node(str(noeud.cle), str(noeud.cle))
            if noeud.gauche is not None:
                dot.edge(str(noeud.cle), str(noeud.gauche.cle), label="L", style="dashed")
                traverse(noeud.gauche)
            if noeud.droite is not None:
                dot.edge(str(noeud.cle), str(noeud.droite.cle), label="R", style="solid")
                traverse(noeud.droite)
        
        traverse(self.racine)
        return dot

## Arbre Auto-adaptatif (Splay-tree) <a class="anchor" id="splaytree"></a>

In [6]:
from graphviz import Digraph

class ArbreBinaire:
    def __init__(self, cle, gauche=None, droite=None):
        self.cle = cle
        self.gauche = gauche
        self.droite = droite

class ArbreSplay:
    def __init__(self):
        self.racine = None
    def _duplique(self,noeud) -> 'ArbreSplay':
        '''ArbreSplay * ArbreBinaire-> ArbreSplay'''
        '''Crée une copie de l'arbre Splay.'''
        if noeud is None:
            return None
        else:
            return ArbreBinaire(noeud.cle, self._duplique(noeud.gauche), self._duplique(noeud.droite))
    #AJOUT
    def ajout_liste(self, cles : list) -> 'ArbreSplay':
        '''ArbreSplay * int list -> ArbreSplay'''
        '''Ajoute une liste de clés dans l'arbre splay.'''
        splay = self
        for cle in cles:
            splay = splay.ajout(cle)
        return splay

    def ajout(self, cle : int) -> 'ArbreSplay':
        '''ArbreSplay * int -> ArbreSplay'''
        '''Ajoute une clé dans l'arbre splay.'''
        splay = ArbreSplay()
        splay.racine = self._duplique(self.racine)
        splay.racine = splay._ajout(splay.racine, cle)
        return splay
        
    def _ajout(self, noeud : ArbreBinaire, cle : int) -> 'ArbreBinaire':
        '''ArbreSplay * ArbreBinaire * int -> ArbreBinaire'''
        '''Ajoute une clé dans l'arbre binaire de recherche et renvoie l'arbre résultant.'''
        if noeud is None:
            return ArbreBinaire(cle)
        elif cle == noeud.cle:
            return noeud
        elif cle < noeud.cle:
            nouveauGauche = self._ajout(noeud.gauche, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, nouveauGauche, noeud.droite)
        else:
            nouveauDroite = self._ajout(noeud.droite, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, noeud.gauche, nouveauDroite)
        return self._splay(nouveauNoeud, cle)
    #SUPPRESSION
    def supprime(self, cle : int) -> 'ArbreSplay':
        '''ArbreSplay * int -> ArbreSplay'''
        '''Supprime une clé de l'arbre splay.'''
        splay = ArbreSplay()
        splay.racine = self._duplique(self.racine)
        splay.racine = splay._supprime(splay.racine, cle)
        return splay
        
    def _supprime(self, noeud : ArbreBinaire, cle : int) -> 'ArbreBinaire':
        '''ArbreSplay * ArbreBinaire * int -> ArbreBinaire'''
        '''Supprime une clé de l'arbre binaire de recherche et renvoie l'arbre résultant.'''
        if noeud is None:
            return noeud
        elif cle < noeud.cle:
            nouveauGauche = self._supprime(noeud.gauche, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, nouveauGauche, noeud.droite)
        elif cle > noeud.cle:
            nouveauDroite = self._supprime(noeud.droite, cle)
            nouveauNoeud = ArbreBinaire(noeud.cle, noeud.gauche, nouveauDroite)
        else:
            if noeud.gauche is None:
                return noeud.droite
            elif noeud.droite is None:
                return noeud.gauche
            
            min_noeud = self._min_noeud(noeud.droite)
            nouveauNoeud = ArbreBinaire(min_noeud.cle, noeud.gauche, self._supprime(noeud.droite, min_noeud.cle))
        return nouveauNoeud
    
    #RECHERCHE
    def recherche(self, cle : int) -> (bool, 'ArbreSplay'):
        '''ArbreSplay * int -> bool * ArbreSplay'''
        '''Recherche une clé dans l'arbre splay et renvoie un booléen indiquant si la clé est présente et l'arbre résultant.'''
        splay = ArbreSplay()
        splay.racine = self._duplique(self.racine)
        splay.racine = splay._splay(splay.racine, cle)
        exist = splay.racine is not None and splay.racine.cle == cle
        return exist, splay


    #     return noeud  # Si aucune rotation n'est effectuée
    def _recherche(self, noeud : ArbreBinaire, cle : int) -> 'ArbreBinaire':
        '''ArbreSplay * ArbreBinaire * int -> ArbreBinaire'''
        '''Recherche une clé dans l'arbre binaire de recherche et renvoie le noeud contenant la clé ou le noeud parent si la clé n'est pas présente.'''
        return self._splay(noeud, cle)
      
    #FONCTIONS UTILES / PRIMITIVES    
    
    def _splay(self, noeud : ArbreBinaire, cle : int) -> 'ArbreBinaire':
        '''ArbreSplay * ArbreBinaire * int -> ArbreBinaire'''
        '''Recherche une clé dans l'arbre binaire de recherche et renvoie le noeud contenant la clé ou le noeud parent si la clé n'est pas présente.'''
        
        if noeud is None or cle == noeud.cle:
            return noeud
        
        if cle < noeud.cle:
            if noeud.gauche is not None:
                if cle < noeud.gauche.cle:
                    noeud.gauche.gauche = self._recherche(noeud.gauche.gauche, cle)
                    noeud = self._rotation_droite(noeud)
                elif cle > noeud.gauche.cle:
                    noeud.gauche.droite = self._recherche(noeud.gauche.droite, cle)
                    if noeud.gauche.droite is not None:
                        noeud.gauche = self._rotation_gauche(noeud.gauche)
                if noeud.gauche is not None:
                    return self._rotation_droite(noeud)
            else:
                return noeud
        else:
            if noeud.droite is not None:
                if cle < noeud.droite.cle:
                    noeud.droite.gauche = self._recherche(noeud.droite.gauche, cle)
                    if noeud.droite.gauche is not None:
                        noeud.droite = self._rotation_droite(noeud.droite)
                elif cle > noeud.droite.cle:
                    noeud.droite.droite = self._recherche(noeud.droite.droite, cle)
                    noeud = self._rotation_gauche(noeud)
                if noeud.droite is not None:
                    return self._rotation_gauche(noeud)
            else:
                return noeud  

    def _rotation_gauche(self, noeud : ArbreBinaire) -> 'ArbreBinaire':
        '''ArbreBinaire -> ArbreBinaire'''
        '''Effectue une rotation gauche sur l'arbre binaire de recherche et renvoie l'arbre résultant.'''
        y = noeud.droite
        noeud.droite = y.gauche
        y.gauche = noeud
        return y

    def _rotation_droite(self, noeud : ArbreBinaire) -> 'ArbreBinaire':
        '''ArbreBinaire -> ArbreBinaire'''
        '''Effectue une rotation droite sur l'arbre binaire de recherche et renvoie l'arbre résultant.'''
        x = noeud.gauche
        noeud.gauche = x.droite
        x.droite = noeud
        return x

    def _min_noeud(self, noeud : ArbreBinaire) -> 'ArbreBinaire':
        '''ArbreBinaire -> ArbreBinaire'''
        '''Renvoie le noeud contenant la clé minimale de l'arbre binaire de recherche.'''
        if noeud is None or noeud.gauche is None:
            return noeud
        return self._min_noeud(noeud.gauche)
    
    def visualiser_arbre(self):
        '''ArbreSplay -> dot'''
        '''Affiche l'arbre binaire'''
        dot = Digraph()
    
        def traverse(noeud):
            if noeud is None:
                return

            dot.node(str(noeud.cle), str(noeud.cle))
            
            if noeud.gauche is not None:
                dot.edge(str(noeud.cle), str(noeud.gauche.cle), label="L", style="dashed")
                traverse(noeud.gauche)
            
            if noeud.droite is not None:
                dot.edge(str(noeud.cle), str(noeud.droite.cle), label="R", style="solid")
                traverse(noeud.droite)
        
        traverse(self.racine)
        return dot

## Arbre Digital (DST) <a class="anchor" id="dst"></a>

In [7]:
from graphviz import Digraph

class ArbreBinaire:
    def __init__(self,cle , gauche = None, droite = None):
        self.cle = cle
        self.gauche = gauche
        self.droite = droite
        
class ArbreDST :
    def __init__(self, dico , c = None):
        self.dico = dico  # Exemple : dico = {'a':'01','b':'11','c':'00','d':'10'}
        self.racine = None            

    def car(self, c , i):
        """DST * str * int -> int/None"""
        """Renvoie le i-ème caractère de l'encodage de c, lorsque c existe dans le dictionnaire, None sinon"""
        for k , v in self.dico.items():
            if k == c and len(v)-1 >= i:
                res= v[i]
                return int(res)
        return None

    def duplique(self , noeud): #A est un arbre binaire
        """DST * ArbreBinaire -> ArbreBinaire"""
        """Renvoie une copie de l'arbre binaire A"""
        #print("passage ?")
        if noeud is None:
            return None
        else:
            return ArbreBinaire(noeud.cle, self.duplique(noeud.gauche), self.duplique(noeud.droite))
        

    def ajout(self, c ): #c est un caractère
        """DST * str -> ArbreBinaire"""
        """Renvoie un arbre binaire contenant le caractère c"""
        return self._ajout(c , 0 , self.racine)

    def _ajout(self, c , i , noeud): #noeud est un arbre binaire
        """DST * str * int * ArbreBinaire -> ArbreBinaire"""
        """Renvoie un arbre binaire contenant le caractère c"""
        if noeud is None:
            return ArbreBinaire(c, None, None)

        if c == noeud.cle:
            return noeud
        if self.car(c,i) == 0 :
            nouveau_D = self.duplique(noeud.droite)
            return ArbreBinaire(noeud.cle, self._ajout(c , i+1 , noeud.gauche), nouveau_D)
        
        if self.car(c,i) == 1 :
            nouveau_G = self.duplique(noeud.gauche)
            return ArbreBinaire(noeud.cle, nouveau_G, self._ajout(c , i+1 , noeud.droite))

        return noeud # Dans le cas ou le caractère n'est pas trouvé



    def recherche(self, c): #c est un caractère
        """DST * str -> bool"""
        """Renvoie True si le caractère c est dans l'arbre, False sinon"""
        return self._recherche( self.racine, c , 0)
    
    def _recherche(self, noeud,  c , i):    #A est un arbre binaire
        """DST * ArbreBinaire * str * int -> bool"""
        """Renvoie True si le caractère c est dans l'arbre, False sinon"""
        if noeud is None: 
            return False
        if c == noeud.cle:    
            return True
        if self.car(c,i) == 0 :  
            return self._recherche(noeud.gauche , c , i+1)
        else :     
            return self._recherche(noeud.droite , c , i+1)
        
        

    def suppression(self, c): #c est un caractère
        """DST * str -> ArbreBinaire"""
        """Supprime le caractère c de l'arbre binaire"""
        return self._suppression(self.racine, c , 0)
    

    def _suppression(self, noeud , c , i): # CBinaire est un caractère binaire, exemple : '0101'
        """DST * ArbreBinaire * str * int -> ArbreBinaire"""
        """Supprime le caractère c de l'arbre binaire"""
        if noeud is None:   #Si l'arbre est vide
            return None
        
        if c == noeud.cle:      #Si le caractère est trouvé, on verifie tous les cas
            #cas si on est sur la racine
            if noeud.gauche is None and noeud.droite is None and self.racine.cle == noeud.cle and self.racine.gauche is None and self.racine.droite is None:
                return None
            if noeud.gauche is None and self.racine.cle == noeud.cle and self.racine.gauche == noeud.gauche:
                return self.duplique(noeud.droite)
            
            if noeud.droite is None and self.racine.cle == noeud.cle and self.racine.droite == noeud.droite:
                return self.duplique(noeud.gauche)
            
            #cas hors de la racine
            if noeud.gauche is None and noeud.droite is None:
                return None
            if noeud.gauche is None:  
                return self.duplique(noeud.droite) 
            if noeud.droite is None:    
                return self.duplique(noeud.gauche)
            else:   #Si le noeud a deux fils
                noeud_min = self.noeud_min(noeud.droite)
                nouveau_G = self.duplique(noeud.gauche)
                return ArbreBinaire(noeud_min.cle, nouveau_G, self._suppression(noeud.droite,noeud_min.cle , i+1) )
            
        if self.car(c,i) == 0 :  
            nouveau_D = self.duplique(noeud.droite)
            return ArbreBinaire(noeud.cle, self._suppression(noeud.gauche , c , i+1), nouveau_D)
        else :#self.car(c,i) == 1:     
            nouveau_G = self.duplique(noeud.gauche)
            return ArbreBinaire(noeud.cle, nouveau_G, self._suppression(noeud.droite , c , i+1))



    def noeud_min(self, noeud ):    #noeud est un arbre binaire
        """DST * ArbreBinaire -> ArbreBinaire"""
        """Renvoie le noeud le plus à gauche, qui est le plus petit noeud de l'arbre binaire A"""
        cour = noeud  
        while (cour.gauche is not None): 
            cour = cour.gauche
        return cour
        

    def construction(self):
        """DST -> DST"""
        """Construit l'arbre binaire à partir du dictionnaire"""
        for c in self.dico:
            self.racine = self.ajout(c)
        return self


    def visualiser_arbre(self):
        """DST -> Digraph"""
        """Renvoie un Digraph qui représente l'arbre binaire"""
        dot = Digraph()

        cpt = 0

        def generer_graphique(noeud):
            nonlocal cpt
            if noeud is not None:
                dot.node(str(noeud.cle))
                if noeud.gauche is not None:
                    dot.edge(str(noeud.cle), str(noeud.gauche.cle) , label = "0" , fontsize="10")
                    generer_graphique(noeud.gauche)
                else:
                    cpt += 1
                    dot.node(f"empty_{cpt}", shape="point", fillcolor="white" , width="0.5")
                    dot.edge(str(noeud.cle), f"empty_{cpt}" , label = "0" , fontsize="10")
                if noeud.droite is not None:
                    dot.edge(str(noeud.cle), str(noeud.droite.cle) , label = "1" , fontsize="10" )
                    generer_graphique(noeud.droite)
                else:
                    cpt += 1
                    dot.node(f"empty_{cpt}", shape="point", fillcolor="white" , width="0.5")
                    dot.edge(str(noeud.cle), f"empty_{cpt}" , label = "1" , fontsize="10")

        generer_graphique(self.racine)
        return dot


    def afficher_arbre(self , arbre, niveau=0):
        if arbre is not None:
            self.afficher_arbre(arbre.droite, niveau + 1)
            if niveau > 0:
                print('   ' * (niveau - 1) + '|--', end='')
            print(str(arbre.cle))
            self.afficher_arbre(arbre.gauche, niveau + 1)

## Arbre Lexicographique <a class="anchor" id="lexico"></a>

In [8]:
from graphviz import Digraph

class ArbreBinaire:
    def __init__(self,cle =None , gauche = None, droite = None):
        self.cle = cle
        self.gauche = gauche
        self.droite = droite
        
class TrieBinaire :
    def __init__(self, dico , c = None):
        self.dico = dico  # Exemple : dico = {'a':'01','b':'11','c':'00','d':'10'}
        if c is None :  #On suppose que c est un caractère de l'alphabet 
            self.racine = None
        else :  
            self.racine = ArbreBinaire(c)

    def car(self, c , i):
        """TrieBin * str * int -> int/None"""
        """Renvoie le i-ème caractère de l'encodage de c, lorsque c existe dans le dictionnaire, None sinon."""
        for k , v in self.dico.items():
            if k == c and len(v)-1 >= i:
                res= v[i]
                return int(res)
        return None

    def ajout(self, c):
        """TrieBin * str -> ArbreBinaire"""
        """Renvoie  l'arbre binaire résultant de l'insertion de c"""
        return self._ajout(c , 0, self.racine)
    

    def duplique(self, A):
        """TrieBin * ArbreBinaire -> ArbreBinaire/None"""
        """Renvoie une copie de l'arbre A, None si A est vide"""
        if A is None:
            return None
        return ArbreBinaire(A.cle, self.duplique(A.gauche), self.duplique(A.droite))
    

    def _ajout(self , c, i , A):
        """"TrieBin * str * int * ArbreBinaire -> ArbreBinaire"""
        """Renvoie le trie binaire en ajoutant le caractère c à l'arbre A, et i est l'indice du caractère c dans l'encodage binaire de c"""
        if A is None:
            return ArbreBinaire(c)
        if A.gauche is None and A.droite is None:
            if c == A.cle:
                return A
            else:
                return self.split( c, A.cle , i)
        if self.car(c, i) == 0:
           nouveau_D = self.duplique(A.droite)
           return ArbreBinaire(A.cle, self._ajout(c, i+1, A.gauche), nouveau_D)
        else:
            nouveau_G = self.duplique(A.gauche)
            return ArbreBinaire(A.cle, nouveau_G , self._ajout(c, i+1, A.droite))


    # Lorsqu'on ajoute un caractère qui à le meme prefixe qu'un autre caractère, on doit diviser le noeud en deux
    def split(self, c1 , c2, i): # c1 est un caractère deja dans l'arbre, c2 est un caractère à ajout
        """TrieBin * str * str * int -> ArbreBinaire"""
        """Retourne le trie binaire contenant c1 et c2"""
        if self.car(c1,i) == self.car(c2,i) == 0:
            return ArbreBinaire( '*' , self.split(c1, c2, i+1), None)
        if self.car(c1,i) == self.car(c2,i) == 1:
            return ArbreBinaire( '*' , None, self.split(c1, c2, i+1))
        if self.car(c1,i) == 0 and self.car(c2,i) == 1:
            return ArbreBinaire( '*' , ArbreBinaire(c1), ArbreBinaire(c2))
        if self.car(c1,i) == 1 and self.car(c2,i) == 0:
            return ArbreBinaire( '*' , ArbreBinaire(c2), ArbreBinaire(c1))
                

    def recherche(self, c):
        """TrieBin * str -> bool"""
        """Renvoie True si c est dans le trie binaire, False sinon"""
        return self._recherche(self.racine, c, 0)
    

    def _recherche(self, arbre, c, i):
        """TrieBin * ArbreBinaire * str * int -> bool"""
        """Renvoie True si c est dans le trie binaire, False sinon"""
        if arbre is None:
            return False
        if arbre.cle == c:
            return True
        if self.car(c, i) == 0:
            return self._recherche(arbre.gauche, c, i + 1)
        else:
            return self._recherche(arbre.droite, c, i + 1)


    def suppression(self, c):
        """TrieBin * str -> TrieBin"""
        """Renvoie le trie binaire après la suppression de c"""
        return self._suppression(self.racine, c, 0)
    

    def _suppression(self, noeud, c, i):
        """TrieBin * ArbreBinaire * str * int -> ArbreBinaire/None"""
        """Renvoie le trie binaire après la suppression de c"""
        res = noeud
        if noeud is None:
            return None
        if noeud.cle == c:
            if noeud.gauche is None and noeud.droite is None:
                return None
            if noeud.gauche is None:
                nouveau_DD = self.duplique(noeud.droite.droite)
                nouveau_DG = self.duplique(noeud.droite.gauche)
                res = ArbreBinaire(noeud.droite.cle, nouveau_DG, nouveau_DD)
                #res = ArbreBinaire(arbre.droite.cle, arbre.droite.gauche, arbre.droite.droite)
            if noeud.droite is None:
                nouveau_GG = self.duplique(noeud.gauche.gauche)
                nouveau_GD = self.duplique(noeud.gauche.droite)
                res = ArbreBinaire(noeud.gauche.cle, nouveau_GG, nouveau_GD)
                #res = ArbreBinaire(arbre.gauche.cle, arbre.gauche.gauche, arbre.gauche.droite)

        if self.car(c, i) == 0:
            nouveau_D = self.duplique(noeud.droite)
            res = ArbreBinaire(noeud.cle, self._suppression(noeud.gauche, c, i + 1), nouveau_D)
        elif self.car(c, i) == 1:
            nouveau_G = self.duplique(noeud.gauche)
            res = ArbreBinaire(noeud.cle, nouveau_G, self._suppression(noeud.droite, c, i + 1))

        # Remettre à jour l'arbre après la suppression
        if res.cle == '*' and res.gauche is None and res.droite is None and self.verifierEnfant(res) == 0:
            # Si l'arbre n'a pas d'enfant
            return None
        
        if res.cle == '*' and res.gauche is not None and res.droite is None and self.verifierEnfant(res) == 1:
            # Si l'arbre a un seul enfant, on peut le remonter 
            return res.gauche   #pas besoin de dupliquer car on a déjà dupliqué l'arbre avant
        
        if res.cle == '*' and res.gauche is None and res.droite is not None and self.verifierEnfant(res) == 1:
            # Si l'arbre a un seul enfant, on peut le remonter
            return res.droite   
        
        return res

    def verifierEnfant(self, noeud):
        """TrieBin * ArbreBinaire -> int"""
        """retourne le nombre d'enfants de l'arbre"""
        if noeud is None:
            return 0
        if noeud.gauche is None and noeud.droite is None:
            return 1
        return self.verifierEnfant(noeud.gauche) + self.verifierEnfant(noeud.droite)



    def construction(self):
        """TrieBin -> TrieBin"""
        """Renvoie le trie binaire construit à partir du dictionnaire"""
        for i in self.dico.keys():
            self.racine = self.ajout(i)
        return self



    def visualiser_arbre(self):
        dot = Digraph()
        compteur = 1

        def generer_graphique(noeud):
            nonlocal dot, compteur
            if noeud is not None:
                tmp = noeud.cle
                cle = str(noeud.cle)
                if '*' in cle:
                    cle = '*'+str(compteur)
                    compteur += 1
                dot.node(cle , label= cle if tmp != '*' else ' ' , width="0.5" if tmp == '*' else '')

            
                if noeud.gauche is not None:
                    if '*' in noeud.gauche.cle:
                        compteur += 1
                        node_g = '*'+str(compteur)
                        dot.edge(cle , node_g , label = "0" , fontsize="10")
                    else:
                        dot.edge(cle , str(noeud.gauche.cle) , label = "0" , fontsize="10")
                    generer_graphique(noeud.gauche)
                elif noeud.gauche is None and noeud.droite is not None:
                    compteur += 1
                    node_g = '*'+str(compteur)
                    dot.node(node_g , label= ' ' , width="0.5")
                    dot.edge(cle , node_g , label = "0" , fontsize="10")


                if noeud.droite is not None:
                    if '*' in noeud.droite.cle:
                        compteur += 1
                        node_d = '*'+str(compteur)
                        dot.edge(cle , node_d , label = "1" , fontsize="10")
                    else:
                        dot.edge(cle , str(noeud.droite.cle) , label = "1" , fontsize="10")
                    generer_graphique(noeud.droite)
                elif noeud.droite is None and noeud.gauche is not None:
                    compteur += 1
                    node_d = '*'+str(compteur)
                    dot.node(node_d , label= ' ' , width="0.5")
                    dot.edge(cle , node_d , label = "1" , fontsize="10")

        generer_graphique(self.racine)
        return dot


    def afficher_arbre(self , arbre, niveau=0):
        if arbre is not None:
            self.afficher_arbre(arbre.droite, niveau + 1)
            if niveau > 0:
                print('   ' * (niveau - 1) + '|--', end='')
            print(str(arbre.cle))
            self.afficher_arbre(arbre.gauche, niveau + 1)

## R-Trie <a class="anchor" id="rtrie"></a>

In [9]:
from graphviz import Digraph

class rtrie:
    def __init__(self, c , v = None , enfant = [None]*26 ): # c : caractere , v : valeur , enfant : liste des enfants
        self.c = c
        self.v = v
        self.enfants = enfant
        
        
class RTrie:
    def __init__(self):
        self.racine = [None]*26
    
    def prem(self , mot): #retourne le caractere en int
        """RTrie * str -> int"""
        """Retourne la position du premier caractere du mot dans l'alphabet."""
        res = ord(mot[0])
        return res-97

    def reste(self , mot):
        """RTrie * str -> str"""
        """Retourne le reste du mot sans le premier caractere."""
        return mot[1:]


    def EnfantSauf(self, A, i):
        """RTrie * rtrie * int -> rtrie[]"""
        """Retourne la liste des enfants de A sauf celui d'indice i."""
        if A is None:
            return []
        res = []

        for cpt in range(len(A.enfants)):
            if cpt == i:
                res.append(None)
            else:
                res.append(self.duplique(A.enfants[cpt]))
        return res



    def SousArbre(self, A, i):
        """RTrie * rtrie * int -> rtrie"""
        """Renvoie une copie du i-eme sous arbre de A"""
        if A.enfants[i] is None :
            return None
        return self.duplique(A.enfants[i])


    def R_Trie(self, c, v, i , L , A): # i : entier , L : liste , A : R-Trie
        """RTrie * str * int * int * rtrie[] * rt -> rtrie"""
        """ Renvoie le trie construit a partir de L en inserant A a la i-eme position,
            en ajoutant la lettre c comme cle et la valeur v
        """
        #Pas besoin de dupliquer L car elle est deja dupliquee dans la methode EnfantSauf
        res = rtrie(c, v)
        res.enfants = L
        res.enfants[i] = A
        return res
    

        
    def duplique(self,A):
        """RTrie * rtrie -> rtrie"""
        """Renvoie une copie de A."""
        if A is None :
            return None
        else:
            res = rtrie(A.c, A.v)
            res.enfants = [None]*26
            for i in range(len(A.enfants)):
                res.enfants[i] = self.duplique(A.enfants[i])
            return res




    def ajout(self, c, v ) : # c : cle, v : valeur
        """RTrie * str * RTrie * -> rtrie"""
        """Ajoute un mot dans l'arbre R-Trie."""
        p = self.prem(c)
        self.racine[p] = self._ajout(c, self.racine[p], v)
        return self.racine



    def _ajout(self, c , A , v ) : # c : cle , A : R-Trie , v : valeur
        """RTrie * str * rtrie * -> rtrie"""
        """Ajoute un mot dans l'arbre R-Trie."""
        if A is None :
            A = rtrie(c)
            
        if len(c) == 1 :
            A.v = v
            return A
        suivant = self.reste(c)[0]
        psuiv = self.prem(suivant)
        return self.R_Trie(c[0], A.v, psuiv, self.EnfantSauf(A, psuiv), self._ajout(self.reste(c),self.SousArbre(A,psuiv),v))


    def recherche(self, mot):
        """RTrie * str -> bool"""
        """Recherche un mot dans l'arbre R-Trie."""
        p = self.prem(mot)
        return self._recherche(mot, self.racine[p])
    

    def _recherche(self, mot, noeud):
        """RTrie * str * rtrie -> bool"""
        """Recherche un mot dans l'arbre R-Trie."""
        if noeud is None:
            return False

        if len(mot) == 1 and noeud.c == mot : # Si nous sommes à la fin du mot et il contient une valeur 
            return noeud.v is not None

        p = self.prem(mot)
        return self._recherche(self.reste(mot), noeud.enfants[p])




    def suppression(self, mot):
        """RTrie * str -> rtrie"""
        """Supprime un mot de l'arbre R-Trie."""
        p = self.prem(mot)
        self.racine[p] = self._suppression(mot, self.racine[p])
        return self.racine
    

    def _suppression(self, mot, noeud):
        """RTrie * str * rtrie -> rtrie"""
        """Supprime un mot de l'arbre R-Trie."""
        res = noeud
        if res is None:
            return None 
        
        if len(mot) < 1:
            return res

        p = self.prem(mot)

        if len(mot) == 1:
            if res.v is not None and mot[0] == res.c:
                res.v = None
            else : 
               res.enfants[p] = self._suppression(self.reste(mot), res.enfants[p])    
                
        elif len(mot) > 1 :
            suivant = self.reste(mot)[0]
            psuiv = self.prem(suivant)
            res = self.R_Trie(mot[0], res.v,  psuiv, self.EnfantSauf(noeud, psuiv), self._suppression(self.reste(mot), self.SousArbre(noeud, psuiv)))
            
        # Cas où le noeud contient une valeur et n'a pas d'enfants
        if mot[0]==res.c and self.contientNone(res.enfants) and res.v is not None:
            return res
        
        # Cas de suppression de noeud inutile
        if mot[0]==res.c and self.contientNone(res.enfants):
            return None

        return res
    

    def contientNone(self, noeud):
        """RTrie * rtrie -> bool"""
        """Vérifie si un noeud contient que des None"""
        for enfant in noeud:
            if enfant is not None:
                return False
        return True



    def afficher_mots(self, noeud=None, mot=''):
        """Affiche tous les mots présents dans l'arbre R-Trie."""
        if noeud is None:
            noeud = self.racine

        for i, enfant in enumerate(noeud):
            if enfant is not None:
                # Si nous sommes à la fin d'un mot, affichons le mot
                if enfant.v is not None:
                    print(mot + chr(i + 97))
                # Sinon, continuons à parcourir l'arbre
                self.afficher_mots(enfant.enfants, mot + chr(i + 97))





    def visualiser_arbre(self):
        dot = Digraph()

        def generer_graphique(arbre, parent_label=''):
            if arbre is not None:
                for enfant in arbre:
                    if enfant is not None:
                        enfant_label = f"{enfant.c}_{id(enfant)}"
                        dot.node(enfant_label, label=f"{enfant.v if enfant.v is not None else ''}")
                        if parent_label is not None:
                            dot.edge(parent_label, enfant_label, label=f" {enfant.c}") 

                        generer_graphique(enfant.enfants, enfant_label)

        # Noeud racine
        dot.node("racine", label="")

        for enfant in self.racine:
            if enfant is not None:
               generer_graphique([enfant], "racine")

        return dot

## Trie Hybride <a class="anchor" id="triehybr"></a>

In [10]:
from graphviz import Digraph

class TrieH:
    def __init__(self , c , inf, eq , sup, v=None ):
        self.c = c
        self.inf = inf
        self.eq = eq
        self.sup = sup
        self.v = v
        
        
class TrieHybride:
    def __init__(self):
        self.racine = None
    
    def lg(self , mot):
        """TH * str -> int"""
        """Retourne la longueur d'un mot"""
        return len(mot)

    def prem(self , mot):
        """TH * str -> str"""
        """Retourne le premier caractère d'un mot"""
        if len(mot) == 0:
            return None
        return mot[0]
    
    def reste(self , mot):
        """TH * str -> str"""
        """Retourne le reste d'un mot sans le premier caractère"""
        return mot[1:]
    
    def duplique(self, A):
        """TH * TrieH -> TrieH"""
        """Duplique un arbre hybride"""
        if A is None:
            return None
        else:
            return TrieH(A.c, self.duplique(A.inf), self.duplique(A.eq), self.duplique(A.sup), A.v)


    def ajout(self, mot, A , v):
        """TH * str * TrieH * int -> TrieH"""
        """Ajoute un mot dans un arbre hybride"""

        if A is None:
            if self.lg(mot) == 1:
                return TrieH(self.prem(mot) , None , None , None , v)
            else:
                return TrieH(self.prem(mot) , None , self.ajout(self.reste(mot), None , v) , None , None)
        
        if self.lg(mot) == 1 :
            n_Inf = self.duplique(A.inf)
            n_Eq = self.duplique(A.eq)
            n_Sup = self.duplique(A.sup)
            return TrieH(A.c , n_Inf , n_Eq , n_Sup , v) 

        else:
            p = self.prem(mot)

            if p == None:
                return A
            if p < A.c:
                # A.inf = self.ajout( mot , A.inf , v)
                n_Eq = self.duplique(A.eq)
                n_Sup = self.duplique(A.sup)
                return TrieH(A.c , self.ajout( mot , A.inf , v) , n_Eq , n_Sup , A.v)
            if p > A.c:
                #print("sup")
                # A.sup = self.ajout( mot , A.sup , v)
                n_Inf = self.duplique(A.inf)
                n_Eq = self.duplique(A.eq)
                return TrieH(A.c , n_Inf , n_Eq , self.ajout( mot , A.sup , v) , A.v)
                #return TrieH(A.c , A.inf , A.eq , self.ajout( mot , A.sup , v) , A.v)
            n_Inf = self.duplique(A.inf)
            n_Sup = self.duplique(A.sup)
            return TrieH(A.c , n_Inf , self.ajout( self.reste(mot) , A.eq , v) , n_Sup , A.v)



    # recherche un mot dans le trie hybride
    def recherche(self, mot, A):
        """TH * str * TrieH -> bool"""
        """Recherche un mot dans un arbre hybride et retourne True si le mot est trouvé, False sinon"""
        if A is None:
            return False
        if self.lg(mot) == 1 and A.c == mot and A.v is not None:
            return True
        p = self.prem(mot)
        if p < A.c:
            return self.recherche(mot, A.inf)
        elif p > A.c:
            return self.recherche(mot, A.sup)
        else:
            return self.recherche(self.reste(mot), A.eq)
        
        
    def suppression(self, mot, noeud):
        """TH * str * TrieH -> TrieH"""
        """Supprime un mot dans un arbre hybride"""
        res = noeud
        if noeud is None:
            return None
        p = self.prem(mot)
        if p < noeud.c:
            n_Eq = self.duplique(noeud.eq)
            n_Sup = self.duplique(noeud.sup)
            res = TrieH(noeud.c, self.suppression(mot, noeud.inf), n_Eq, n_Sup, noeud.v)
            
        elif p > noeud.c:
            n_Inf = self.duplique(noeud.inf)
            n_Eq = self.duplique(noeud.eq)
            res = TrieH(noeud.c, n_Inf, n_Eq, self.suppression(mot, noeud.sup), noeud.v)
            
        elif p == noeud.c:
            if self.lg(mot) == 1:
                if noeud.v is not None:
                    noeud.v = None
                    
            else:
                n_Inf = self.duplique(noeud.inf)
                n_Sup = self.duplique(noeud.sup)
                res = TrieH(noeud.c, n_Inf, self.suppression(self.reste(mot), noeud.eq), n_Sup, noeud.v)

        # Cas de suppression de la racine
        if noeud.c == self.racine.c and noeud.eq is None and noeud.inf is None and noeud.sup is None and noeud.v is None and self.racine.eq == noeud.eq and self.racine.inf == noeud.inf and self.racine.sup == noeud.sup and self.racine.v == noeud.v:
            return None

        # Cas de suppression hors de la racine
        if self.prem(mot) == res.c and res.eq is None and res.inf is None and res.sup is None and res.v is None:
            return None
            
        if res.inf is None and res.eq is None and res.sup is None and res.v is None:
            return None
        
        if res.inf is not None and res.eq is None and res.sup is not None:
            return self.Fusion(res.inf, res.sup)
  
        if res.inf is not None and res.eq is None and res.sup is None:
            return res.inf 
        if res.inf is None and res.eq is None and res.sup is not None:
            return res.sup  
        return res 





    # Fusionner deux arbres
    def Fusion(self, A, B): 
        """TH * TrieH * TrieH -> TrieH"""
        """Fusionne deux arbres hybrides"""
        if A is None:
            return B
        if B is None:
            return A
        if A is not None and B is not None:
            if A.c < B.c:
                n_Inf = self.duplique(A.inf)
                n_Eq = self.duplique(A.eq)
                return TrieH(A.c, n_Inf, n_Eq, self.Fusion(A.sup, B), A.v)
                                
            if A.c > B.c:
                n_Sup = self.duplique(B.sup)
                n_Eq = self.duplique(B.eq)
                return TrieH(B.c, self.Fusion(A, B.inf), n_Eq, n_Sup, B.v)
               
            if A.c == B.c:
                n_SupA = self.duplique(A.sup)
                n_InfA = self.duplique(A.inf)
                return TrieH(A.c, n_InfA, self.Fusion(A.eq, B.eq), n_SupA, A.v)
                
    

       
    def visualiser_arbre(self):
        dot = Digraph()
        fontsize = "10"

        def generer_graphique(A, dot=dot):
            if A is not None:
                dot.node(str(id(A)), label=f"{A.c} {({str(A.v)}) if A.v is not None else ''}")

                if A.inf is not None:
                    dot.node(str(id(A.inf)), label=f"{A.inf.c} (Inf)")
                    dot.edge(str(id(A)), str(id(A.inf)) , label="Inf", fontsize=fontsize )
                    generer_graphique(A.inf)

                if A.eq is not None:
                    dot.node(str(id(A.eq)), label=f"{A.eq.c} (Eq)")
                    dot.edge(str(id(A)), str(id(A.eq)) , label="Eq", fontsize=fontsize )
                    generer_graphique(A.eq)

                if A.sup is not None:
                    dot.node(str(id(A.sup)), label=f"{A.sup.c} (Sup)")
                    dot.edge(str(id(A)), str(id(A.sup)) , label="Sup", fontsize=fontsize )
                    generer_graphique(A.sup)

        generer_graphique(self.racine)
        return dot